In [1]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFileApply
from cognite.client.exceptions import CogniteException
from IPython.display import display

from tests.auth_utils import get_toolkit_client

In [2]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [3]:
SPACE = "sp_upgrade"

In [4]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Cascading Delete

In [5]:
file = FileMetadataWrite(
    external_id="ts_case2_cascading_delete",
    name="Case 2 Cascading Delete",
)

In [6]:
# Ensure the file does not exist before running the test
client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)

In [7]:
created, _ = client.files.create(file)
created

,value
external_id,ts_case2_cascading_delete
name,Case 2 Cascading Delete
labels,[]
id,3246816829946726
created_time,2025-07-18 08:05:41.596000
last_updated_time,2025-07-18 08:05:41.596000
uploaded,False


In [8]:
filecontent = b"This is filecontent"

In [9]:
client.files.upload_content_bytes(filecontent, external_id=created.external_id)

,value
external_id,ts_case2_cascading_delete
name,Case 2 Cascading Delete
id,3246816829946726
created_time,2025-07-18 08:05:41.596000
last_updated_time,2025-07-18 08:05:41.596000
uploaded,False


In [10]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    display(updated)

,value
external_id,ts_case2_cascading_delete
name,Case 2 Cascading Delete
id,3246816829946726
created_time,2025-07-18 08:05:41.596000
last_updated_time,2025-07-18 08:05:46.255000
uploaded,True
uploaded_time,2025-07-18 08:05:44.174000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."


In [11]:
cognite_file = CogniteFileApply(
    space=SPACE,
    external_id=file.external_id,
    name=file.name,
)

In [12]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,ts_case2_cascading_delete,2,True,2025-07-18 08:05:49.586,2025-07-10 05:59:32.286


In [13]:
try:
    client.files.delete(external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    print("\033[92mFile deleted successfully\033[0m")

Error 403 Resource not found. This may also be due to insufficient access rights.


In [14]:
deleted = client.data_modeling.instances.delete(cognite_file.as_id()).nodes
deleted

[NodeId(space='sp_upgrade', external_id='ts_case2_cascading_delete')]

In [15]:
try:
    client.files.retrieve_multiple(external_ids=[file.external_id])
except CogniteException as e:
    print(f"\033[91mError\033[0m {e!s}")
else:
    print("\033[92mFile  still exists\033[0m")

Error Not found: [{'externalId': 'ts_case2_cascading_delete'}]
The API Failed to process some items.
Successful (2xx): []
Unknown (5xx): []
Failed (4xx): [{'externalId': 'ts_case2_cascading_delete'}, ...]
